In [ ]:
import time
import numpy as np
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook

output_notebook()

my_figure = figure(plot_width=800, plot_height=400)
test_data = ColumnDataSource(data=dict(x=[0], y=[0]))
line = my_figure.line("x", "y", source=test_data)
handle = show(my_figure, notebook_handle=True)

new_data=dict(x=[0], y=[0])
x = []
y = []

step = 0
step_size = 0.1  # increment for increasing step
max_step = 10  # arbitrary stop point for example
period = 1  # in seconds (simulate waiting for new data)
n_show = 10  # number of points to keep and show


while step < max_step:
    x.append(step)
    y.append(np.random.rand())
    new_data['x'] = x = x[-n_show:]  # prevent filling ram
    new_data['y'] = y = y[-n_show:]  # prevent filling ram

    test_data.stream(new_data, n_show)

    push_notebook(handle=handle)
    step += step_size
    time.sleep(period)

In [2]:
"""
  Need to run `ping google.com >> /tmp/google.txt` in another terminal.
"""
import re
from bokeh.plotting import *
from bokeh.objects import GlyphRenderer

MAX_HISTORY = 1000
output_server("ping")

def tail_generator(filename="/tmp/google.txt", interval=0.5):
    """
    Reads file and then `tails` it -- checking at interval seconds
    Adapted from: http://code.activestate.com/recipes/157035-tail-f-in-python/
    """
    with open(filename, 'r') as f:
        while True:
            where = f.tell()
            line = f.readline()
            if not line:
                time.sleep(interval)
                f.seek(where)
            else:
                yield line

def parse_line(line):
    """Find sequence and rtt -- return as int, float or return None"""
    regex = re.compile(r'icmp_seq=(?P<seq>[0-9]+) .* time=(?P<time>[0-9.]+) ms')
    m = regex.search(line)
    if not m:
        return None
    return int(m.groupdict()["seq"]), float(m.groupdict()["time"])

x = [-1]
y = [0]
line(x,y)
#show()

renderer = [r for r in curplot().renderers if isinstance(r, GlyphRenderer)][0]
ds = renderer.data_source

f = tail_generator()
for line_ in f:
    
    data = parse_line(line_)
    if data is None:
        continue
    x.append(data[0])
    y.append(data[1])
    
    if len(x) > MAX_HISTORY:
        x = x[-MAX_HISTORY:]
    if len(y) > MAX_HISTORY:
        y = y[-MAX_HISTORY:]
    ds.data["x"] = x
    ds.data["y"] = y
   
    ds._dirty = True
    session().store_obj(ds)
    time.sleep(.01)

ModuleNotFoundError: No module named 'bokeh.objects'